<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/temp_hosting/workinprogress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import os
from datetime import date, timedelta

# Function to get Nifty 50 tickers
def get_nifty50_tickers():
    # As of November 2023, I don't have updated information on Nifty 50 constituents
    # You'll need to update this list with the current Nifty 50 tickers
    # You can find this information on the NSE website or other financial sources
    nifty50_tickers = [
        # "RELIANCE.NS", "TCS.NS", ... (add other Nifty 50 tickers here)
    ]
    return nifty50_tickers

# Download data for each ticker and save to separate CSV files
nifty50_tickers = get_nifty50_tickers()

for ticker in nifty50_tickers:
    try:
        data = yf.download(ticker)  # Download data without specifying start or end date
        data.to_csv(f"{ticker}.csv")  # Save to CSV with ticker as filename
        print(f"Data for {ticker} saved successfully.")
    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")

In [ ]:
# Function to get Nifty 50 tickers
def get_nifty50_tickers():
    # Update this list with the current Nifty 50 tickers
    nifty50_tickers = [
        # "RELIANCE.NS", "TCS.NS", ... (add other Nifty 50 tickers here)
    ]
    return nifty50_tickers

# Get today's date and the date of the last available data (assuming it's yesterday)
today = date.today()
last_data_date = today - timedelta(days=1)

# Get list of CSV files in the directory
csv_files = [f for f in os.listdir() if f.endswith(".csv")]

# Update each CSV file
for filename in csv_files:
    try:
        # Read existing data
        df = pd.read_csv(filename, index_col="Date", parse_dates=True)

        # Get the last date in the existing data
        last_date_in_df = df.index[-1].date()

        # Check if update is needed
        if last_date_in_df < last_data_date:
            # Download new data from yfinance
            ticker = filename.split(".")[0]  # Extract ticker from filename
            new_data = yf.download(ticker, start=last_date_in_df + timedelta(days=1), end=today)

            # Append new data to existing DataFrame
            df = pd.concat([df, new_data])
            df.to_csv(filename, index=True)  # Save updated DataFrame to CSV
            print(f"Updated data for {ticker}")
        else:
            print(f"Data for {ticker} is already up-to-date.")

    except Exception as e:
        print(f"Error updating file {filename}: {e}")

In [ ]:
import pandas as pd
import os
from datetime import date, timedelta

# Define output columns
output_columns = ["Name", "Daily", "1 Week", "1 Month", "YTD", "1 Year", "3 Years"]

# Function to calculate returns
def calculate_returns(data, periods):
    returns = {}
    for period, days in periods.items():
        if len(data) >= days:
            returns[period] = (data["Close"][-1] - data["Close"][-days]) / data["Close"][-days]
        else:
            returns[period] = "-"  # Not enough data
    return returns

# Get list of CSV files in the directory
csv_files = [f for f in os.listdir() if f.endswith(".csv")]

results = []
for filename in csv_files:
    try:
        # Read CSV data
        data = pd.read_csv(filename, index_col="Date", parse_dates=True)

        # Calculate returns for different periods
        periods = {"Daily": 1, "1 Week": 7, "1 Month": 30, "YTD": (date.today() - date(date.today().year, 1, 1)).days, "1 Year": 365, "3 Years": 365 * 3}
        returns = calculate_returns(data, periods)

        # Get stock name from filename (assuming filename format is "TICKER.csv")
        stock_name = filename.split(".")[0]

        # Create row for output
        row = [stock_name] + [returns[period] for period in output_columns[1:]]
        results.append(row)

    except Exception as e:
        print(f"Error processing file {filename}: {e}")

# Create DataFrame and save to CSV
output_df = pd.DataFrame(results, columns=output_columns)
output_df.to_csv("Screener Results.csv", index=False)
print("Results saved to Screener Results.csv")